# Yaozhong Zhang project 1 

In [1]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#here we go
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline


## lane tracking

In [5]:
def process_image(image):

    # Read it as a gray image
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    kernel_size = 3
    blur_gray = cv2.GaussianBlur(gray,(kernel_size,kernel_size),0)

    #Canny edge function
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(blur_gray,low_threshold, high_threshold)
    
    
    mask = np.zeros_like(edges)
    ignore_mask_color = 255

    imshape = image.shape
    vertices = np.array([[(80,imshape[0]),(460, 310), (imshape[1]-440, 310), (imshape[1]-70,imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)

    #to show the result so far (masking)
    line_image = np.copy(image)*0

#     color_edges = np.dstack((masked_edges,masked_edges,masked_edges))
#     combo = cv2.addWeighted(color_edges,1.0,line_image,1.0,0)
    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1.1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 3     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 30 #minimum number of pixels making up a line
    max_line_gap = 10   # maximum gap in pixels between connectable line segments
    #line_image = np.copy(image)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)

    #show hough transformation results
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    color_edges = np.dstack((masked_edges,masked_edges,masked_edges))

    leftline=[]
    rightline=[]
    leftintercept=[]
    rightintercept=[]

    for line in lines:
        for x1,y1,x2,y2 in line:
            if 3>((y2-y1)/(x2-x1)) and ((y2-y1)/(x2-x1))>0.33:
                rightline.append((y2-y1)/(x2-x1))
                rightintercept.append(y2-((y2-y1)/(x2-x1))*x2)
            elif -3<((y2-y1)/(x2-x1)) and ((y2-y1)/(x2-x1))<-0.33:
                leftline.append((y2-y1)/(x2-x1))
                leftintercept.append(y2-((y2-y1)/(x2-x1))*x2)

    # get left/right average slope

    sum = 0
    for i in leftline:
        sum = sum +i
    a_leftline = sum/len(leftline)

    sum = 0
    for i in rightline:
        sum = sum +i
    a_rightline = sum/len(rightintercept)

    sum = 0
    for i in leftintercept:
        sum = sum +i
    a_leftintercept = sum/len(leftintercept)

    sum = 0
    for i in rightintercept:
        sum = sum +i
    a_rightintercept = sum/len(rightintercept)
    
    y_1 = 320
    y_2 = 540
    x_l1 = (y_1-a_leftintercept)/a_leftline
    x_l2 = (y_2-a_leftintercept)/a_leftline
    x_r1 = (y_1-a_rightintercept)/a_rightline
    x_r2 = (y_2-a_rightintercept)/a_rightline
    
    cv2.line(image,(int(x_l1),y_1),(int(x_l2),y_2),(255,0,0),5)
    cv2.line(image,(int(x_r1),y_1),(int(x_r2),y_2),(0,0,255),5)
    
    image_final = image
    
    return image_final

In [6]:
white_output = 'test_videos_output/solidYellowLeft.mp4'
# clip1 = VideoFileClip("test_videos/challenge.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidYellowLeft.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:14<00:00, 47.89it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 14.8 s


In [9]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))